In [ ]:
import pandas as pd
pd.set_option('display.precision',13)
import io
import re
from google.colab import files

In [ ]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [127]:
uploaded_pagseguro = drive.CreateFile({'id':'[YOUR ID]'})   # id pagseguro
uploaded_pagseguro.GetContentFile('[YOUR FILE NAME]')        # name pagseguro
uploaded_paypal = drive.CreateFile({'id':'[YOUR ID]'})   # id paypal
uploaded_paypal.GetContentFile('[YOUR FILE NAME]')        # name paypal
uploaded_shopify = drive.CreateFile({'id':'[YOUR ID]'})   # id shopify
uploaded_shopify.GetContentFile('[YOUR FILE NAME]')        # name shopify
df_shopify = pd.read_csv('[YOUR FILE NAME]')
df_paypal = pd.read_csv('[YOUR FILE NAME]')
df_pagseguro = pd.read_csv('[YOUR FILE NAME]',index_col=False, encoding='latin-1', sep=';')
df_pagseguro['Ref_Transacao'] = df_pagseguro['Ref_Transacao'].apply(str).replace('c|.\0|\.1|\.2|\.3|\.4|\.5','').map(lambda x: str(x)[:-2])
df_paypal['Número da fatura'] = df_paypal['Número da fatura'].str.replace('c|.\0|\.1|\.2|\.3|\.4|\.5','')
df_shopify['Payment Reference'] = df_shopify['Payment Reference'].str.replace('c|.\0|\.1|\.2|\.3|\.4|\.5','')
df_paypal.rename(columns={'Número da fatura':'Payment Reference'}, inplace=True)
df_paypal.rename(columns={'Tarifa':'tax_provider'}, inplace=True)
df_pagseguro.rename(columns={'Ref_Transacao':'Payment Reference'}, inplace=True)
df_pagseguro.rename(columns={'Valor_Taxa':'tax_provider'}, inplace=True)
df_paypal['tax_provider'] = df_paypal['tax_provider'].str.replace('-','')
df_paypal['tax_provider'] = df_paypal['tax_provider'].str.replace(',','.').astype(float)
df_pagseguro['tax_provider'] = df_pagseguro['tax_provider'].str.replace(',','.').astype(float)
df_providers = df_paypal[df_paypal['Payment Reference'].apply(lambda x: len(str(x))>3)][['Payment Reference','tax_provider']]
df_providers = df_providers.append(df_pagseguro[df_pagseguro['Payment Reference'].apply(lambda x: len(str(x))>3)][['Payment Reference','tax_provider']])
df_result = pd.merge(df_shopify,df_providers,how='left',left_on='Payment Reference',right_on='Payment Reference')
df_final = df_result[df_result['Financial Status'] == 'paid'][['Name','Email','Vendor','Lineitem name','Total','Discount Amount','Payment Method','Payment Reference','tax_provider']]
df_final['tax_shopify'] = df_final.Total * 0.02 # % of shopify
df_final['tax_zeeplo'] = (df_final.Total - df_final.tax_shopify - df_final.tax_provider)*.1 # % of partner
df_final['creator_Amount'] = df_final.Total - df_final.tax_shopify - df_final.tax_zeeplo - df_final.tax_provider

In [128]:
  df_final.to_csv(r'final_data.csv')